# Introduccion a Langchain

## Componentes esenciales

---


In [ ]:
!pip install ollama -q

In [ ]:
import ollama 

In [ ]:
from langchain_community.llms import Ollama 

In [ ]:
llm = Ollama(model="llama3.2")

In [ ]:
respuesta = llm.invoke("Cuentame un chiste")
print(respuesta)